In [0]:
import numpy as np
import matplotlib.pyplot as plt

#******************************************************
#                 NN forward pass
#******************************************************

def NN(X,W1,W2,W3):
    #datanum,dim=X.shape
    # Input X is row-wise samples
    # Output Y is also row-wise samples

    v1 = np.matmul(W1, X.T)
    v1 = v1.T
    Y1 = Sigmoid(v1)

    v2 = np.matmul(W2, Y1.T)
    v2 = v2.T
    Y2 = Sigmoid(v2)

    v3 = np.matmul(W3, Y2.T)
    Y = softmax(v3)
    Y = Y.T

    return Y


# ******************************************************
#                 NN backward pass
# ******************************************************

def BackpropCEMulti2Hiddens(W1, W2, W3, X, D, alpha):
    # alpha is a learning rate
    datanum, dim = X.shape

    for k in range(datanum):
        x = np.array(X[k, :], ndmin=2).T
        d = np.array(D[k, :], ndmin=2).T

        # NN forward
        v1 = np.matmul(W1, x)
        y1 = Sigmoid(v1)

        v2 = np.matmul(W2, y1)
        y2 = Sigmoid(v2)

        v3 = np.matmul(W3, y2)
        y = softmax(v3)

        # NN backward
        e = d - y
        delta = e

    # ***********************************
    #
    #     Fill in the blank
    #
    # ***********************************

        e1 = np.matmul(W2.T, delta2)
        delta1 = y1 * (1 - y1) * e1
        
        # NN update

        dW1 = alpha * np.matmul(delta1, x.T)  # delta rule
        W1 = W1 + dW1

        dW2 = alpha * np.matmul(delta2, y1.T)
        W2 = W2 + dW2

        dW3 = alpha * np.matmul(delta, y2.T)
        W3 = W3 + dW3

    return W1, W2, W3

# ******************************************************
#                 NN Logistic sigmoid
# ******************************************************
def Sigmoid(x):
    # return 1.0 / (1.0 + np.exp(-x))
    return np.exp(-np.logaddexp(0, -x))


#******************************************************
#                 NN softmax
#******************************************************


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)


  
  
# ******************************************************
#                Upload data
# ******************************************************
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
    
data = np.load('data_3_MLP_multiple_python.npz')
X = data['X']
Y = data['Y']
datanum,inputdim=X.shape
_,outputdim=Y.shape
print('Number of input data:%d, Input dim:%d' %(datanum, inputdim))
print('Number of output data:%d, Output dim:%d' %(datanum, outputdim))



In [0]:
print(Y)

In [0]:
# ******************************************************
#                Train MLP
# ******************************************************
datanum,inputdim=X.shape
_,outputdim=Y.shape
X=np.column_stack((X,np.ones([datanum,1])))
D = Y
learing_rate = 0.001


epochnum = 10000
ERR = np.zeros([epochnum,1])
hiddennum1 = 20
hiddennum2 = 20

W1 = 0.1*(2*np.random.random((hiddennum1,inputdim+1)) - 1)
W2 = 0.1*(2*np.random.random((hiddennum2,hiddennum1)) - 1)
W3 = 0.1*(2*np.random.random((outputdim,hiddennum2)) - 1)


for epoch in range(epochnum):
    W1, W2, W3 = BackpropCEMulti2Hiddens(W1, W2, W3, X, D,learing_rate)
    Yhat = NN(X,W1,W2,W3)
    ERR[epoch] = 1/2*np.trace(np.matmul((D-Yhat).T,(D-Yhat)))/datanum
    if epoch%100 == 0:
       print("epoch = %d, Err=%f" % (epoch,ERR[epoch]))


plt.plot(ERR, 'r')
plt.show()



In [0]:
# ******************************************************
#                Test MLP
# ******************************************************

def plotNN2D(X,Y,W1,W2,W3):

    C1 = np.where(Y[:,0] == 1)[0]
    C2 = np.where(Y[:,1] == 1)[0]
    C3 = np.where(Y[:,2] == 1)[0]
    C = (C1,C2,C3)
    colors = ("red", "green","black")

    for c,color in zip(C, colors):
        plt.scatter(X[c,0], X[c,1], alpha=1.0, c=color)


    step = 0.025
    x_axis = np.arange(0.0, 1.0+step, step)
    y_axis = np.arange(0.0, 1.0+step, step)
    X_mesh, Y_mesh = np.meshgrid(x_axis, y_axis)

    Z_mesh0 = np.zeros(X_mesh.shape)
    Z_mesh1 = np.zeros(X_mesh.shape)
    Z_mesh2 = np.zeros(X_mesh.shape)


    for x in range(X_mesh.shape[0]):
        for y in range(Y_mesh.shape[0]):
            input = np.array([X_mesh[x][y], Y_mesh[x][y], 1],ndmin=2)
            output = NN(input,W1,W2,W3)
            Z_mesh0[x][y] = output[0][0]
            Z_mesh1[x][y] = output[0][1]
            Z_mesh2[x][y] = output[0][2]
 
    plt.contour(x_axis, y_axis, Z_mesh0, ([0.69, 0.71]), colors="red", linewidths=3)
    plt.contour(x_axis, y_axis, Z_mesh1, ([0.69, 0.71]), colors="green", linewidths=3)
    plt.contour(x_axis, y_axis, Z_mesh2, ([0.69, 0.71]), colors="black", linewidths=3)


    plt.show()


#   ***             Plot
plotNN2D(X,Y,W1,W2,W3)